In [1]:
import HelperFuncs as hfuncs
import numpy as np

We're going to first try training a CNN on the individual images.
We will be using binary cross entropy across the 17 regions.

In [21]:
import HelperFuncs as hfuncs
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.data_utils import Sequence
import h5py
import os

BATCH_SIZE = 1
FINAL_WIDTH = 400
FINAL_HEIGHT = 600
CHANNELS = 1
ZONES = 17
ANGLES = 16
#Create directories for sequencer function if they don't exist
for d in ['temp/train_scan/','temp/test_scan/','temp/val_scan/']:
    if not os.path.isdir(d):
        print("Created directory: {}".format(d))
        os.makedirs(d)
        
class ScanSequencer(Sequence):
    idx_dict={}
    
    def __init__(self,num_batches,bucket_name,mode="train_scan"):
        self.num_batches = num_batches
        self.bucket_name = bucket_name
        self.mode = mode
        self.key_id, self.secret_key = hfuncs.GetAWSCredentials()
        self.mode = mode
        self.angles = np.arange(0,64,64//ANGLES)
    def __len__(self):
        return self.num_batches
    def on_epoch_end(self):
        pass
    def __getitem__(self,idx):
        #Get Client
        client = hfuncs.GetAWSClient(self.key_id,self.secret_key)
        bucket = client.Bucket(self.bucket_name)
        
        #Initialize vectors
        X_train = np.zeros((BATCH_SIZE,ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
        y_train = np.zeros((BATCH_SIZE,ZONES))
        
        j=0
        for i in range(idx*BATCH_SIZE,(idx+1)*BATCH_SIZE):
            #Download batch at index
            path = "temp/{}/batch_{}.hdf5".format(self.mode,i)
            key = "{}/batch_{}.hdf5".format(self.mode,i)
            bucket.download_file(Key=key,Filename=path)
        
            f = h5py.File(path,"r")
            try:
                X_train[j,:,:,:,:] = f['/image'].value[self.angles,:,:,:]
                y_train[j,:] = f['/labels'].value
                j += 1
            finally:
                f.close()
                os.remove(path) 
        return X_train, y_train
class SingleScanSequencer(Sequence):
    idx_dict={}
    
    def __init__(self,num_batches,bucket_name,mode="train_scan",target_zone = 0):
        self.num_batches = num_batches
        self.bucket_name = bucket_name
        self.mode = mode
        self.key_id, self.secret_key = hfuncs.GetAWSCredentials()
        self.mode = mode
        self.target_zone = target_zone
        self.angles = np.arange(0,64,64//ANGLES)
    def __len__(self):
        return self.num_batches
    def on_epoch_end(self):
        pass
    def __getitem__(self,idx):
        #Get Client
        client = hfuncs.GetAWSClient(self.key_id,self.secret_key)
        bucket = client.Bucket(self.bucket_name)
        
        #Initialize vectors
        X_train = np.zeros((BATCH_SIZE,ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
        y_train = np.zeros((BATCH_SIZE,1))
        
        j=0
        for i in range(idx*BATCH_SIZE,(idx+1)*BATCH_SIZE):
            #Download batch at index
            path = "temp/{}/batch_{}.hdf5".format(self.mode,i)
            key = "{}/batch_{}.hdf5".format(self.mode,i)
            bucket.download_file(Key=key,Filename=path)
        
            f = h5py.File(path,"r")
            try:
                X_train[j,:,:,:,:] = f['/image'].value[self.angles,:,:,:]
                y_train[j,:] = f['/labels'].value[self.target_zone]
                j += 1
            finally:
                f.close()
                os.remove(path) 
        return X_train, y_train



In [22]:
from twilio.rest import Client
import configparser
from keras.callbacks import Callback

class SMSNotifier(Callback):
    def on_epoch_end(self,epoch,logs=None):
        #Execute every other epoch
        if epoch % 2 == 0:
            #Get config credentials
            config = configparser.ConfigParser()
            config.read('twilio.conf')
            account_sid = config['DEFAULT']['AccountID']
            auth_token = config['DEFAULT']['AuthToken']
            #Get client
            client = Client(account_sid, auth_token)
            #Create message
            if logs is not None:
                message = "Epoch {} complete. Loss: {} Val_loss: {} ".format(epoch,
                                                                             logs.get('loss'),
                                                                             logs.get('val_loss'))
            else:
                message = "Epoch {} complete. No loss data available.".format(epoch)
            #Sendmessage
            message = client.messages.create(
                to="+16178884129", 
                from_="+18572142288",
                body=message)
        else:
            pass

In [23]:
from keras import backend as K
import tensorflow as tf


def weighted_binary_crossentropy(y_true, y_pred):
    x = 0.10 #Approximate percentage of positives in each of the 17 zones
    POS_ADJ = 0.5/x
    NEG_ADJ = 0.5/(1-x)
    n_values = BATCH_SIZE
    elems = (tf.unstack(y_true,num=n_values,axis=0)) 
    adj = tf.map_fn(lambda x:tuple([tf.cond(tf.equal(x[i],1.),lambda:POS_ADJ,lambda: NEG_ADJ) for i in range(n_values)]),
                    elems, 
                    dtype = tuple([tf.float32 for i in range(n_values)]) )
    adj = tf.stack(adj,axis=0)
    return K.mean(tf.multiply(adj,K.binary_crossentropy(y_true,y_pred)),axis=-1)

In [27]:
#Build pre-trained V2 model
import numpy as np
from keras.layers import Input,Flatten,Dense,Concatenate,Dropout,concatenate,GlobalMaxPool2D
from keras.models import Model
from datetime import datetime
from keras.callbacks import TensorBoard,EarlyStopping,ModelCheckpoint,ReduceLROnPlateau
from keras.optimizers import Adam
from keras.metrics import binary_accuracy
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import LSTM
from keras.losses import binary_crossentropy
from keras.applications import inception_resnet_v2 as resv2
from keras.layers.core import Lambda
import tensorflow as tf

def ToRGB(x):
    max_v = tf.reduce_max(x)
    min_v = tf.reduce_min(x)
    max_rgb = tf.constant(255,dtype=x.dtype)
    x = tf.floordiv(tf.multiply(tf.subtract(x,min_v),max_rgb),tf.subtract(max_v,min_v))
    return x

def ToNewShape(x):
    ndim = len(x.shape)
    if ndim == 5:
        return tf.reverse(tf.transpose(x,[0,1,3,2,4]),[-3])
    elif ndim == 4:
        return tf.reverse(tf.transpose(x,[0,2,1,3]),[-3])
    else:
        raise ValueError("Unexpected number of dims!")

def getModel(lstm_dim = 500):
    #Single model image
    input_img = Input(shape=(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    
    #preprocess and extract channels
    input_img_pp = Lambda(ToRGB)(input_img)
    input_img_pp = Lambda(resv2.preprocess_input)(input_img_pp)
    input_img_pp = Lambda(ToNewShape)(input_img_pp)
    
    #Extract 3 channels in order to use pre-trained weights
    extract_channels = resv2.conv2d_bn(input_img_pp,
                  filters=3,
                  kernel_size=1,
                  strides=(3,2),
                  padding='same',
                  activation='relu',
                  use_bias=False,
                  name=None)
    #Load resnet
    resnet = resv2.InceptionResNetV2(include_top=False,
                          weights='imagenet',
                          input_tensor=None,
                          input_shape=(FINAL_HEIGHT,FINAL_WIDTH,3),
                          pooling='max')
    for l in resnet.layers:
        l.trainable=False

    #Take off top
    reduced_resnet = Model(resnet.input,resnet.get_layer('mixed_6a').output)
    output = reduced_resnet(extract_channels)
    output = GlobalMaxPool2D()(output)
    print("Max pool output {}".format(output.shape))
    intermediate_model = Model(input_img,output)

    #Time distributed model
    input_scan = Input(shape=(ANGLES,FINAL_HEIGHT,FINAL_WIDTH,CHANNELS))  
    sequenced_model = TimeDistributed(intermediate_model)(input_scan)
    #model = Model(input_scan,sequenced_model)
    #print(model.input_shape,model.output_shape)
    #One lstm layer for now

    lstm = LSTM(lstm_dim,recurrent_dropout=0.20)(sequenced_model)

    #Finally, 17 dense layers connected to the output
    output_nodes = []
    for i in range(ZONES):
        output_nodes.append(Dense(1,activation='sigmoid')(lstm))

    out = concatenate(output_nodes)
    del resnet
    #complete model
    try:
        return Model(input_scan, out)
    finally:
        del intermediate_model,reduced_resnet
def getSingleModel(lstm_dim = 250):
    #Single model image
    input_img = Input(shape=(FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))
    
    #preprocess and extract channels
    input_img_pp = Lambda(ToRGB)(input_img)
    input_img_pp = Lambda(resv2.preprocess_input)(input_img_pp)
    input_img_pp = Lambda(ToNewShape)(input_img_pp)
    
    #Extract 3 channels in order to use pre-trained weights
    extract_channels = resv2.conv2d_bn(input_img_pp,
                  filters=3,
                  kernel_size=1,
                  strides=(3,2),
                  padding='same',
                  activation='relu',
                  use_bias=False,
                  name=None)
    #Load resnet
    resnet = resv2.InceptionResNetV2(include_top=False,
                          weights='imagenet',
                          input_tensor=None,
                          input_shape=(FINAL_HEIGHT,FINAL_WIDTH,3),
                          pooling='max')
    for l in resnet.layers:
        l.trainable=False

    #Take off top
    reduced_resnet = Model(resnet.input,resnet.get_layer('mixed_6a').output)
    output = reduced_resnet(extract_channels)
    output = GlobalMaxPool2D()(output)
    print("Max pool output {}".format(output.shape))
    intermediate_model = Model(input_img,output)

    #Time distributed model
    input_scan = Input(shape=(ANGLES,FINAL_WIDTH,FINAL_HEIGHT,CHANNELS))  
    sequenced_model = TimeDistributed(intermediate_model)(input_scan)
    #model = Model(input_scan,sequenced_model)
    #print(model.input_shape,model.output_shape)
    #One lstm layer for now

    lstm = LSTM(lstm_dim,recurrent_dropout=0.20)(sequenced_model)

    #Finally, 1 dense layers
    out = Dense(1,activation='sigmoid')(lstm)
    del resnet
    #complete model
    try:
        return Model(input_scan, out)
    finally:
        del intermediate_model,reduced_resnet

In [31]:
from keras.models import load_model
#optimizer
lr = 0.001
beta1 = 0.9
beta2 = 0.999
description = "Pretrained model"
lstm_dim = 1000
recurrent_model = getSingleModel(20)
#recurrent_model = load_model('check_points/2017_9_26_21_ScanModel_00-0.70.hdf5')

Max pool output (?, 1088)


In [32]:
print("Compiling model...")
adam = Adam(lr,beta_1=beta1,beta_2=beta2)
recurrent_model.compile(optimizer=adam,
                          metrics=[binary_accuracy,binary_crossentropy],
                         loss= weighted_binary_crossentropy)

#Tensorboard
x = datetime.today()
stamp = "{}-{}-{}_{}:{}:{}_lr-{}_beta1-{}_beta2-{}_lstmDim-{}_ANG-{}_{}".format(x.year,x.month,
                                                     x.day,x.hour,x.minute,
                                                     x.second,lr,beta1,beta2,lstm_dim,ANGLES,description)
tensorboard = TensorBoard(log_dir="logs/{}".format(stamp),histogram_freq=0,batch_size=BATCH_SIZE,
                          write_grads=False,write_images=False)
#Model checkpoint
check_point_dir = 'check_points/'
if not os.path.isdir(check_point_dir):
    os.makedirs(check_point_dir)    
chkpt = ModelCheckpoint(os.path.join(check_point_dir,"{}_{}_{}_{}_".format(x.year,x.month,x.day,x.hour) + "ScanModel_{epoch:02d}-{val_loss:.2f}.hdf5"),
                       monitor='val_binary_crossentropy',
                       verbose=1,
                       save_best_only=True)
#Reduce learning rate on plateau
reduce_lr = ReduceLROnPlateau(monitor='val_binary_crossentropy',
                             factor=0.2,
                             patience=1,
                             verbose=1,
                             min_lr=0.0001,
                             cooldown = 4)
#Notifications
notify = SMSNotifier()

#Early stopping callback
estop = EarlyStopping(monitor='val_binary_crossentropy',min_delta=0.0001,patience=5)

#Generators and fit
print("Initializing generators...")
#Bucket with clean data
UPLOAD_BUCKET = 'cleandhsdata' #bucket where clean data was stored
key_id, secret_key = hfuncs.GetAWSCredentials()
client = hfuncs.GetAWSClient(key_id,secret_key)
bucket = client.Bucket(UPLOAD_BUCKET)

#Initialize train sequencer
mode ="train_scan"
num_batches_train = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
#num_batches = 660//BATCH_SIZE
train_seq = SingleScanSequencer(num_batches_train,UPLOAD_BUCKET,mode=mode,target_zone=0)

#Initialize validation sequencer
mode = "val_scan"
num_batches_val = (sum([1 if "{}/".format(mode) in k.key else 0 for k in bucket.objects.all()])-1)//BATCH_SIZE #train,test,val root directories have their own keys
val_seq = SingleScanSequencer(num_batches_val,UPLOAD_BUCKET,mode=mode,target_zone=0)



print("Beginning training...")
try:
    hist,model = recurrent_model.fit_generator(train_seq,
                                       steps_per_epoch=num_batches_train,
                                       #steps_per_epoch=5,
                                       epochs=100,
                                       validation_data = val_seq,
                                       validation_steps = num_batches_val,
                                       #validation_steps = 5,
                                       callbacks=[tensorboard,chkpt,reduce_lr,notify,estop],
                                      use_multiprocessing =False,workers=1)
finally:
    os.system("aws ec2 stop-instances --instance-ids i-0172c75d2de9bad71")

Compiling model...
Initializing generators...
Beginning training...
Epoch 1/100
891/891 [==============================] - 1919s - loss: 0.7807 - binary_accuracy: 0.2144 - binary_crossentropy: 0.7228 - val_loss: 0.7771 - val_binary_accuracy: 0.1295 - val_binary_crossentropy: 0.7320
Epoch 2/100
891/891 [==============================] - 1924s - loss: 0.7394 - binary_accuracy: 0.1156 - binary_crossentropy: 0.7381 - val_loss: 0.7746 - val_binary_accuracy: 0.1295 - val_binary_crossentropy: 0.7541
Epoch 3/100
890/891 [============================>.] - ETA: 1s - loss: 0.7397 - binary_accuracy: 0.1157 - binary_crossentropy: 0.7587Epoch 00002: val_binary_crossentropy did not improve

Epoch 00002: reducing learning rate to 0.00020000000949949026.
891/891 [==============================] - 2161s - loss: 0.7394 - binary_accuracy: 0.1156 - binary_crossentropy: 0.7588 - val_loss: 0.7741 - val_binary_accuracy: 0.1295 - val_binary_crossentropy: 0.7594
Epoch 4/100
891/891 [============================

TypeError: 'History' object is not iterable